<a href="https://colab.research.google.com/github/arinjay-singh/econ3916-statistical-machine-learning/blob/main/Class%202/class2_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import os

api_key = os.environ["FRED_API_KEY"]
# print(f"API Key: {api_key}")

In [2]:
# install the FRED API library
!pip install fredapi

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
from fredapi import Fred

# Authenticate
# Replace the string below with YOUR unique API key from fred.stlouisfed.org
fred = Fred(api_key=api_key)

In [34]:
# fetch the data series
print("Fetching data from FRED...")
wages = fred.get_series('AHETPI')
cpi = fred.get_series('CPIAUCSL')

# combine into a DataFrame
df = pd.DataFrame({'Nominal_Wage': wages, 'CPI': cpi})

# drop missing values (The wage data starts later than CPI data)
df = df.dropna()

Fetching data from FRED...


In [35]:
df.head(10)

,Nominal_Wage,CPI
1964-01-01,2.50,30.94
1964-02-01,2.50,30.91
1964-03-01,2.51,30.94
1964-04-01,2.52,30.95
1964-05-01,2.52,30.98
1964-06-01,2.53,31.01
1964-07-01,2.54,31.02
1964-08-01,2.55,31.05
1964-09-01,2.56,31.08
1964-10-01,2.55,31.12


In [36]:
df.tail(10)

,Nominal_Wage,CPI
2025-01-01,30.80,319.086
2025-02-01,30.91,319.775
2025-03-01,30.97,319.615
2025-04-01,31.05,320.321
2025-05-01,31.14,320.580
2025-06-01,31.26,321.500
2025-07-01,31.35,322.132
2025-08-01,31.45,323.364
2025-09-01,31.52,324.368
2025-11-01,31.73,325.031
